## Department of Computer Science, University of York
### DATA: Introduction to Data Science

## Task 1: Domain Analysis  (5 marks)

Given the business domain and the data overview presented (in the assessment paper), provide a brief description of

* the business problem and its significance to the relevant sector;
* the link between the business problem and the field of data science;
* the main areas of investigation; and
* potential ideas and solutions.


**Word Limit:** 300 words

**Write your answer here (text cell(s) to be used, as appropriate)**

In [1]:
### Write your answer here (code cell(s) to be used, as appropriate)



----
----


## Task 2: Database Design (25 marks)


Having understood the business domain, present a conceptual design in the form of an entity-relationship (ER) model that would be helpful in creating a database for the bank.

The bank data currently exists in the form of a csv file called *BankRecords.csv*, provided on VLE (path given in page 5, assessment paper). This file has all the existing records. The table available in the csv file is unnormalised. The information about its different columns is given in Tables 1 and 2 (in the assessment paper).

Following the standard principles of database normalisation, normalise the given table (*BankRecords.csv*) to a database schema that has minimum redundancies. Then, using the designed schema, create an SQLite database.

Your answer should include the SQL statements needed to accomplish this step. Your submission should also include the created SQLite database file.

Your answer should clearly cover the following:
* Any assumptions you are making about the given scenario;
* The designated keys, existing relationships, and identified functional dependencies;
* The steps followed and justifications for the decisions made.

**World Limit**: 500 words. This limit applies only to the explanations. There is no limit on any associated code/SQL statements or figures.

**Write your answer here (text cell(s) to be used, as appropriate)**

For the ERM diagram, see `DATA Essay.jpg`.

![ERM Diagram](DATA Essay.jpg)

In [2]:
import sqlite3
from pathlib import Path

In [20]:
### Write your answer here (code cell(s) to be used, as appropriate)

# todo not nulls
# todo constraints
# todo the following classifier fields could be made integers:
#     statementFrequency
#     loanStatus
#     transType
#     dispType
#     cardType

sql_create = """
CREATE TABLE Account (
    accountID INTEGER PRIMARY KEY,
    statementFrequency TEXT,  -- frequency
    creationDate TEXT
);
CREATE TABLE Loan (
    loanID INTEGER PRIMARY KEY,
    dispID INTEGER,
    loanDate TEXT,
    loanAmount REAL,
    loanDuration INTEGER,
    loanPayments INTEGER,
    loanStatus TEXT,
    FOREIGN KEY (dispID) REFERENCES Disposition(dispID)
);
CREATE TABLE StandingOrder (
    orderID INTEGER PRIMARY KEY,
    dispID INTEGER,
    bankTo STRING,
    accountTo INTEGER,
    orderAmount REAL,
    paymentType STRING,
    FOREIGN KEY (dispID) REFERENCES Disposition(dispID)
);
CREATE TABLE BankTransaction (  -- Transaction is a reserved word
    transID INTEGER PRIMARY KEY,
    dispID INTEGER,
    transDate TEXT,
    transType TEXT,
    operation TEXT,
    transAmount REAL,
    balance REAL,
    transDetail TEXT,
    partnerBank TEXT,
    partnerAccount INTEGER,
    FOREIGN KEY (dispID) REFERENCES Disposition(dispID)
);
CREATE TABLE Client (
    clientID INTEGER PRIMARY KEY,
    cityID INTEGER,  -- a1
    birthDate TEXT,  -- birth_number
    gender INTEGER,  -- birth_number
    FOREIGN KEY (cityID) REFERENCES City(cityID)
);
CREATE TABLE Disposition (
    dispID INTEGER PRIMARY KEY,
    accountID INTEGER,
    clientID INTEGER,
    dispType TEXT,
    FOREIGN KEY (accountID) REFERENCES Account(accountID),
    FOREIGN KEY (clientID) REFERENCES Client(clientID)
);
CREATE TABLE CreditCard (
    cardID INTEGER PRIMARY KEY,
    dispID INTEGER,
    cardType TEXT,
    cardIssued TEXT,
    FOREIGN KEY (dispID) REFERENCES Disposition(dispID)
);
CREATE TABLE City (
    cityID INTEGER PRIMARY KEY,  -- a1
    cityName TEXT,
    region TEXT,
    inhabitants INTEGER,
    muns0 INTEGER,
    muns500 INTEGER,
    muns2000 INTEGER,
    muns10000 INTEGER,
    noAreas INTEGER,
    ratioUrban REAL,
    avgSalary REAL,
    unemployment1995 REAL,
    unemployment1996 REAL,
    entrepeneurs INTEGER,
    crimes1995 INTEGER,
    crimes1996 INTEGER  -- a16
);
"""

sqlf = Path("BankRecords.db")


def create_db():
    print("Deleting database")
    try:
        sqlf.unlink()
    except FileNotFoundError:
        print("Database didn't exist")
    else:
        print("Deleted database")
    
    print("Creating database")
    con = sqlite3.connect(sqlf)
    with con:
        con.executescript(sql_create)
    con.close()
    print("Done")

create_db()

Deleting database
Deleted database
Creating database
Done


In [4]:
import pandas as pd

In [72]:
%%time

# load dataframe
DATE_FORMAT_CSV = "%y%m%d"

csvf = Path("BankRecords.csv")

print("Loading csv")

df = pd.read_csv(
    csvf,
    parse_dates=[
        "creation_date",
        "loan_date",
        "trans_date",
        "card_issued",
    ],
    date_format={
        "creation_date": DATE_FORMAT_CSV,
        "loan_date": DATE_FORMAT_CSV,
        "trans_date": DATE_FORMAT_CSV,
        "card_issued": f"{DATE_FORMAT_CSV} 00:00:00",  # they lied about this one
    },
    low_memory=False,
)

Loading csv
Done in 17.6s


In [74]:
df.head()

,account_id,frequency,creation_date,loan_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status,order_id,...,a7,a8,a9,a10,a11,a12,a13,a14,a15,a16
0,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
1,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
2,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
3,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910
4,1,Monthly,1995-03-24,NaN,NaT,NaN,NaN,NaN,NaN,29401.0,...,2,1,4,65.3,8968,2.83,3.35,131,1740.0,1910


In [75]:
print(df.shape)
print(df.dtypes)
df.columns

(2243458, 46)
account_id                  int64
frequency                  object
creation_date      datetime64[ns]
loan_id                   float64
loan_date          datetime64[ns]
loan_amount               float64
loan_duration             float64
loan_payments             float64
loan_status                object
order_id                  float64
bank_to                    object
account_to                float64
order_amount              float64
payment_type               object
trans_id                    int64
trans_date         datetime64[ns]
trans_type                 object
operation                  object
trans_amount              float64
balance                   float64
trans_detail               object
partner_bank               object
partner_account           float64
disp_id                     int64
client_id                   int64
disp_type                  object
card_id                   float64
card_type                  object
card_issued        datetime64[ns]


Index(['account_id', 'frequency', 'creation_date', 'loan_id', 'loan_date',
       'loan_amount', 'loan_duration', 'loan_payments', 'loan_status',
       'order_id', 'bank_to', 'account_to', 'order_amount', 'payment_type',
       'trans_id', 'trans_date', 'trans_type', 'operation', 'trans_amount',
       'balance', 'trans_detail', 'partner_bank', 'partner_account', 'disp_id',
       'client_id', 'disp_type', 'card_id', 'card_type', 'card_issued',
       'birth_number', 'a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9',
       'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16'],
      dtype='object')

In [90]:
%%time

# ISO 8601 date
# https://www.sqlite.org/lang_datefunc.html
DATE_FORMAT_SQL = "%Y-%m-%d"


# reset db for testing
create_db()


def filter_df(columns: list[str]):
    return df.filter(columns).groupby(columns[0]).first()


print("Inserting data")
con = sqlite3.connect(sqlf)
cur = con.cursor()

accounts = filter_df(
    ["account_id", "frequency", "creation_date"]
)
# no inplace option for this 3':
accounts["creation_date"] = accounts["creation_date"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO Account VALUES (?, ?, ?);",
    accounts.itertuples()
)

loans = filter_df(
    ["loan_id", "disp_id", "loan_date", "loan_amount", "loan_duration", "loan_payments", "loan_status"]
)
loans["loan_date"] = loans["loan_date"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO Loan VALUES (?, ?, ?, ?, ?, ?, ?);",
    loans.itertuples()
)

standing_orders = filter_df(
    ["order_id", "disp_id", "bank_to", "account_to", "order_amount", "payment_type"]
)
cur.executemany(
    "INSERT INTO StandingOrder VALUES (?, ?, ?, ?, ?, ?);",
    standing_orders.itertuples()
)

transactions = filter_df(
    ["trans_id", "disp_id", "trans_date", "trans_type", "operation", "trans_amount", "balance", "trans_detail", "partner_bank", "partner_account"]
)
transactions["trans_date"] = transactions["trans_date"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO BankTransaction VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?);",
    transactions.itertuples()
)
con.commit()  # particularly large one

clients = filter_df(
    ["client_id", "a1", "birth_number"]
)
# bool mask of all rows with MM+50
clients["gender"] = ((clients["birth_number"] // 100) % 100) > 12
# remove offset
clients.loc[clients["gender"], "birth_number"] -= 5000
# parse date
clients["birth_number"] = pd.to_datetime(clients["birth_number"], format=DATE_FORMAT_CSV)
clients["birth_number"] = clients["birth_number"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO Client VALUES (?, ?, ?, ?)",
    clients.itertuples()
)

dispositions = filter_df(
    ["disp_id", "account_id", "client_id", "disp_type"]
)
# dispo elysium
cur.executemany(
    "INSERT INTO Disposition VALUES (?, ?, ?, ?);",
    dispositions.itertuples()
)

credit_cars = filter_df(
    ["card_id", "disp_id", "card_type", "card_issued"]
)
credit_cars["card_issued"] = credit_cars["card_issued"].dt.strftime(DATE_FORMAT_SQL)
cur.executemany(
    "INSERT INTO CreditCard VALUES (?, ?, ?, ?);",
    credit_cars.itertuples()
)

cities = filter_df(
    [f"a{i}" for i in range(1, 16+1)]
)
cur.executemany(
    "INSERT INTO City VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
    cities.itertuples()
)

cur.close()
con.commit()
con.close()
print("Inserting data complete")

Deleting database
Deleted database
Creating database
Done
Inserting data
Inserting data complete
CPU times: user 4.84 s, sys: 287 ms, total: 5.13 s
Wall time: 5.47 s


----
----


## Task 3: Research Design (25 Marks)

Using the database designed in Task 2, design and implement **five** potential modelling solutions to achieve the aim of the Data Intelligence team. You need to provide clear justifications about the techniques selected in the context of the 'problem in hand'. Your design must consist of a combination of inferential statistics, supervised learning algorithms, and unsupervised learning algorithms, and include **at least one** of those techniques. Finally, your modelling solutions should be of sufficient complexity, combining information from multiple tables from the database built in Task 2, as appropriate. Your answer should clearly show the queries made to the database. If amendments are made to the database, the commands should be clearly included in your answer.

Your answer should clearly cover the following:
* Any assumptions you are making about the given scenario;
* Any data processing and data integrity steps you would undertake to make the data fit for purpose;
* Which technique(s) you would apply for each solution and why;
* An evaluation of the techniques applied in terms of the accuracy of their results (or any other suitable evaluation measure);
* Algorithmic parameters should be adequately stated and discussed;
* A discussion of ethical considerations arising from the solutions selected.

**World Limit**: 500 words. This limit applies only to the explanations. There is no limit on any associated code or figures.

**Write your answer here (text cell(s) to be used, as appropriate)**

In [ ]:
### Write your answer here (code cell(s) to be used, as appropriate)

----
----

## Task 4: Experimental Results and Analysis (25 Marks)

Given the **five** modelling solutions implemented above, analyse, discuss and present your findings to the key stakeholders of the bank.

Your answer should clearly cover the following:
* Present your findings in a clear and concise manner;
* Discuss your results in the context of the selected solution;
* Discuss how these results can help the bank in performing customer risk assessment and establishing customer retention strategies;
* Present the limitations (if any) of your solutions in a clear and concise manner.

**World Limit**: 500 words. This limit applies only to the explanations. There is no limit on any associated code or figures.

**Write your answer here (text cell(s) to be used, as appropriate)**

In [ ]:
### Write your answer here (code cell(s) to be used, as appropriate)


----
----

## Task 5: Conclusion (10 Marks)

Given the insights derived from Tasks 1-4, provide a conclusion that clearly covers the following:
* A summary of the main points;
* A discussion of the significance of your results;
* Any recommendation(s) resulting from your analysis;
* Any overall ethical considerations arising from the data analysis of this business domain.

**World Limit**: 300 words.

**Write your answer here (text cell(s) to be used, as appropriate)**

In [ ]:
### Write your answer here (code cell(s) to be used, as appropriate)


----
----

## Overall Academic Quality (10 Marks)
10 marks are allocated for the clarity and cohesiveness of your answers (both text and code) across all tasks with appropriate, relevant and effective analysis and presentation of the results.

## Deliverables

You should submit the following to the submission point on the teaching portal:

1. the SQLite database produced in Task 2;
2. the completed Jupyter notebook (both .ipynb and HTML files) that also includes the SQL statements (Task 2), the research design and its implementation (Task 3), and the analysis and presentation of your results (Task 4);
3. any figures or diagrams that are included in your answers in the Jupyter notebook.

For each task where text is required, we have provided guidelines above on the suggested word counts. Exceeding the word count will result in any work beyond the word count being disregarded when assessing.